### Trading Testing

Testing delta probability for weeklies and monthlies
    - Does delta match probability of occurrence
    - Check Kelly criterion for long straddles/delta neutral back ratios
        - To calculate expected losses and gains, use options pricing module to interpolate prices at given percentage
        moves and use the normal pdf as the probability weightings
    
Check earnings returns post announcement factors
    - Factors:
        - Number of times beaten earnings (Dummy Variable)
        - Consecutive earnings beats
        - Consecutive earnings upsets
        - 3 Month Trend before earnings
        - YTD Trend
        - Momentum of monthly returns (20 day, 60 day)


In [2]:
import datetime as dt

import numpy as np
import pandas as pd
from pandas_datareader.data import Options
from py_vollib.black_scholes_merton.implied_volatility import *
# ts = TimeSeries(key='5HZEUI5AFJB06BUK',output_format='pandas')
import py_vollib
from py_vollib.black_scholes_merton.implied_volatility import *
from py_vollib.black_scholes_merton.greeks.analytical import *
import plotly
import os
import pandas_market_calendars as mcal
import json
from pandas.io.json import json_normalize
import urllib.request as req
import time
nyse = mcal.get_calendar('NYSE')
from helpers import *

from scipy.stats import norm as norm
# '''
# functions list:
    
#     maturities(dt.datetime()) --> [float(front_wgt), float(back_wgt)]
    
#     optionslam_scrape(str[ticker]) --> dict[earnings]
    
#     yahoo_table_parse(str[raw_html_table]) --> DataFrame[ticker]
    
#     yahoo_earnings(dt.datetime()) --> DataFrame[earnings_on_date]
    
#     fundamentals(str[ticker]) --> DataFrame[stock_fundamentals]
    
#     get_fundas(list[ticker_lst]) --> DataFrame[stock_fundamentals]
    
#     historical_data(str[ticker], int[day_number], int[rolling_window], outsize[str]) --> DataFrame[daily_stock_data]
    
#     current_volatility(list[ticker_lst], int[roll]) --> DataFrame[stock_volatilities]
    
#     all_options(str[ticker], bool[greeks]) --> DataFrame[options_chains]
    
#     earnings_condor(str[ticker], int[max_gap], int[dte_thresh], float[|money_thresh| <= 1]) -- DataFrame[condors], DataFrame[puts], DataFrame[calls]
    
#     write_excel(str[filename], list[str[sheetnames]], list[dataframes]) --> void()
    
#     curr_stock_data(str[ticker]) --> DataFrame[stock_info]
    
#     curr_batch_quotes(list_of_string[tickers]) --> DataFrame[stock_info]

#     past_earnings(str[ticker]) --> DataFrame[earnings_info]

#     earnings_history(str[ticker]) --> [DataFrame[earnings_estimate], DataFrame[past_earnings], DataFrame[earnings_estimate_change]]
    
#     av_data(str[ticker]) --> DataFrame[ticker_open, ticker_close]

#     av_batch(list_of_str[tickers]) --> DataFrame[tickers_closes]

#     check_mkt_corr(int[corr_rolling_window],int[plot_window]) --> DataFrame[rolling_corr]

#     vvix_check() --> DataFrame[VVIX Data]

#     earnings_longs(list_of_str[ticker], float[bid_ask_spread]) --> DataFrame[option_chains]

#     all_options_v2(str[ticker], int[dte_ub], int[dte_lb], float[moneyness]) --> DataFrame[option_chains]

#     yahoo_options_query(str[ticker], int[dte_ub], int[dte_lb]) --> DataFrame[option_chains]

#     greek_calc(DataFrame[option_chain], str[prem_price_use], str[day_format], float[interest_rate], float[dividend_rate])

#     price_sim(DataFrame[options_df], float[price_change], float[vol_change], int[days_change], str[output = 'All'],
#               str[skew = 'flat'], str[day_format = 'trading'], float[interest_rate = 0.0193], float[dividend_rate = 0],
#               float[prem_price_use = 'Mid'])


#     position_sim(DataFrame[position_df], list_of_int[holdings], int[shares],
#                  float[price_change], float[vol_change], int[dte_change], str[output = 'All'],
#                  str[skew = 'flat'], str[prem_price_use = 'Mid'], str[day_format = 'trading'], 
#                  float[interest_rate = 0.0193], float[dividend_rate = 0])

#     yahoo_fundamentals(list_of_str[tickers]) --> DataFrame[fundamentals]

# '''

In [2]:
# Pulling Indices Data

spx = pd.read_csv('spx.csv', index_col = 0)[['Open','Close']]
spx.columns = ['spx_{0}'.format(x) for x in spx.columns]

vix = pd.read_csv('vix.csv', index_col = 0)[['Open','Close']]
vix.columns = ['vix_{0}'.format(x) for x in vix.columns]

vvix = pd.read_csv('vvix.csv', index_col = 0)[['Close']]
vvix.columns = ['vvix']

yields = pd.read_csv('https://www.quandl.com/api/v3/datasets/USTREASURY/YIELD.csv?api_key=dzmzEExntfap7SNx5p6t', index_col = 0)
df = pd.concat([spx,yields[yields.columns[1:]]], axis = 1).dropna()
df.index = pd.to_datetime(df.index)
df[df.columns] = df[df.columns].apply(pd.to_numeric, errors='coerce')

In [3]:
# Creating full calendar

nyse_dates = nyse.schedule(start_date=df.index[0].date().strftime('%Y-%m-%d'), end_date=df.index[-1].date().strftime('%Y-%m-%d'))
date_idx = [x.date() for x in mcal.date_range(nyse_dates, frequency='1D')]

date_df = pd.DataFrame(columns = ['Trading_Day'],
                       index = range(len(date_idx)))
date_df.index = date_idx
date_df.index = pd.to_datetime(date_df.index)
date_df['Trading_Day'] = True

start_date = dt.datetime(1993,12,1).date()

full_cal = []
curr_date = start_date
while curr_date != dt.datetime.today().date():
    full_cal.append(curr_date)
    curr_date = curr_date + dt.timedelta(days = 1)
    
calendar_df = pd.DataFrame(columns = ['Weekday'],
                           index = full_cal)
calendar_df.index = pd.to_datetime(calendar_df.index)
calendar_df['Weekday'] = calendar_df.index.weekday
date_df = calendar_df.join(date_df).fillna(False)

In [4]:
spx_check = date_df.join(df)
spx_check = spx_check[spx_check['Weekday'].isin([0,1,2,3,4])]
spx_check['Week'] = spx_check.index.week
spx_check['Month'] = spx_check.index.month
spx_check['Year'] = spx_check.index.year
spx_check['Month_Expiry'] = False
spx_check['New_Month'] = False
spx_check['Week_Start'] = False
spx_check['Week_End'] = False

# Marking Monthly Expirations for SPX and Following Trading Day
for idx, row in spx_check[['Month','Year']].drop_duplicates().iterrows():
    curr_eigth_day = dt.date(row['Year'],row['Month'],7)
    curr_third_fri = curr_eigth_day - dt.timedelta(dt.date(row['Year'],row['Month'],3).weekday()) + dt.timedelta(14)
    curr_third_thurs = curr_eigth_day - dt.timedelta(dt.date(row['Year'],row['Month'],4).weekday()) + dt.timedelta(14)
    curr_third_wed = curr_eigth_day - dt.timedelta(dt.date(row['Year'],row['Month'],5).weekday()) + dt.timedelta(14)
    
    # Marking monthly expiry days: Third Friday of every month, otherwise third thursday
    if spx_check.loc[curr_third_fri,'spx_Open'] != np.nan:
        spx_check.loc[curr_third_fri,'Month_Expiry'] = True
        
    elif spx_check.loc[curr_third_thurs,'spx_Open'] != np.nan:
        spx_check.loc[curr_third_thurs,'Month_Expiry'] = True
        
    elif spx_check.loc[curr_third_wed,'spx_Open'] != np.nan:
        spx_check.loc[curr_third_wed,'Month_Expiry'] = True
        
# Marking Monthly Starting
for idx, row in spx_check[spx_check['Month_Expiry'] == True].iterrows():
    following_week = spx_check[spx_check.index > idx].head()
    for idx_2, row_2 in following_week.iterrows():
        if row_2['Trading_Day']:
            spx_check.loc[idx_2, 'New_Month'] = True
            break
            
# Setting Weekly Tags

for idx, row in spx_check[['Week','Year']].drop_duplicates().iterrows():
    current_week = spx_check[(spx_check['Week'] == row['Week']) &
                             (spx_check['Year'] == row['Year']) &
                             (spx_check['Trading_Day'] == True)]
    week_start = current_week.head(1)
    week_end = current_week.tail(1)
    
    try:
        spx_check.loc[week_start.index[0].date(), 'Week_Start'] = True
        spx_check.loc[week_end.index[0].date(), 'Week_End'] = True
    except:
        continue

spx_check['Monthlies'] = spx_check['Month_Expiry'] | spx_check['New_Month']
spx_monthly_check = spx_check[spx_check['Monthlies'] == True]

spx_check['Weeklies'] = spx_check['Week_Start'] | spx_check['Week_End']
spx_weekly_check = spx_check[(spx_check['Weeklies'] == True) &
                             (spx_check['Year'] > 1993) &
                             (spx_check['Year'] < 2018)]

# Filling in missing values
spx_monthly_check.spx_Open = spx_monthly_check.spx_Open.fillna((spx_monthly_check.spx_Open.shift() + 
                                                                spx_monthly_check.spx_Open.shift(-1))/2)
spx_monthly_check.spx_Close = spx_monthly_check.spx_Close.fillna((spx_monthly_check.spx_Close.shift() + 
                                                                  spx_monthly_check.spx_Close.shift(-1))/2)

spx_weekly_check.spx_Open = spx_weekly_check.spx_Open.fillna((spx_weekly_check.spx_Open.shift() + 
                                                              spx_weekly_check.spx_Open.shift(-1))/2)
spx_weekly_check.spx_Close = spx_weekly_check.spx_Close.fillna((spx_weekly_check.spx_Close.shift() + 
                                                                spx_weekly_check.spx_Close.shift(-1))/2)

C:\Users\Fang\Anaconda3\lib\site-packages\pandas\core\generic.py:2999: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [5]:
spx_weekly_check = spx_weekly_check[['spx_Open','spx_Close','Week_Start','Week_End','Year'] + list(yields.columns[1:])]
spx_weekly_check['Week_Return'] = spx_weekly_check.spx_Close/spx_weekly_check.spx_Open.shift(1) - 1
spx_weekly_check = spx_weekly_check[spx_weekly_check['Week_End'] == True]
spx_weekly_check['Up_Moves'] = (spx_weekly_check['Week_Return'] > 0.005)*1
spx_weekly_check['Down_Moves'] = (spx_weekly_check['Week_Return'] < -0.005)*1
spx_weekly_check['No_Moves'] = 1 - spx_weekly_check['Up_Moves'] - spx_weekly_check['Down_Moves']

weekly_counts = {'Ups':[],'Downs':[],'Flats':[]}
weekly_tally = spx_weekly_check[['Up_Moves','Down_Moves', 'No_Moves']]

for yr in spx_weekly_check.Year.drop_duplicates():
    weekly_counts['Ups'].append(weekly_tally[weekly_tally.index.year == yr].sum()['Up_Moves'])
    weekly_counts['Downs'].append(weekly_tally[weekly_tally.index.year == yr].sum()['Down_Moves'])
    weekly_counts['Flats'].append(weekly_tally[weekly_tally.index.year == yr].sum()['No_Moves'])
    
weekly_df = pd.DataFrame(weekly_counts, index = spx_weekly_check.Year.drop_duplicates())
weekly_df = weekly_df.drop([2002,2003,2004,2005,2006])
weekly_df_pct = pd.concat([(weekly_df['Ups']/weekly_df.sum(axis = 1)),
                           (weekly_df['Downs']/weekly_df.sum(axis = 1)),
                           (weekly_df['Flats']/weekly_df.sum(axis = 1))],axis = 1)
weekly_df_pct.columns = ['UpMove','DownMove','NoMove']
weekly_df_pct

,UpMove,DownMove,NoMove
Year,,,
1994,0.365385,0.288462,0.346154
1995,0.576923,0.173077,0.250000
1996,0.519231,0.326923,0.153846
1997,0.461538,0.365385,0.173077
1998,0.547170,0.320755,0.132075
1999,0.519231,0.365385,0.115385
2000,0.423077,0.538462,0.038462
2001,0.442308,0.442308,0.115385
2007,0.480769,0.346154,0.173077


In [6]:
spx_monthly_check = spx_monthly_check[['spx_Open','spx_Close','New_Month','Month_Expiry','Year'] + list(yields.columns[1:])]
spx_monthly_check['Month_Return'] = spx_monthly_check.spx_Close/spx_monthly_check.spx_Open.shift(1) - 1
spx_monthly_check = spx_monthly_check[spx_monthly_check['Month_Expiry'] == True]
spx_monthly_check['Up_Moves'] = (spx_monthly_check['Month_Return'] > 0.01)*1
spx_monthly_check['Down_Moves'] = (spx_monthly_check['Month_Return'] < -0.01)*1
spx_monthly_check['No_Moves'] = 1 - spx_monthly_check['Up_Moves'] - spx_monthly_check['Down_Moves']

monthly_counts = {'Ups':[],'Downs':[],'Flats':[]}
monthly_tally = spx_monthly_check[['Up_Moves','Down_Moves', 'No_Moves']]

for yr in spx_monthly_check.Year.drop_duplicates():
    monthly_counts['Ups'].append(monthly_tally[monthly_tally.index.year == yr].sum()['Up_Moves'])
    monthly_counts['Downs'].append(monthly_tally[monthly_tally.index.year == yr].sum()['Down_Moves'])
    monthly_counts['Flats'].append(monthly_tally[monthly_tally.index.year == yr].sum()['No_Moves'])
    
monthly_df = pd.DataFrame(monthly_counts, index = spx_monthly_check.Year.drop_duplicates())
monthly_df = monthly_df.drop([2002,2003,2004,2005,2006])
monthly_df_pct = pd.concat([(monthly_df['Ups']/monthly_df.sum(axis = 1)),
                           (monthly_df['Downs']/monthly_df.sum(axis = 1)),
                           (monthly_df['Flats']/monthly_df.sum(axis = 1))],axis = 1)
monthly_df_pct.columns = ['UpMove','DownMove','NoMove']
monthly_df_pct

,UpMove,DownMove,NoMove
Year,,,
1993,0.000000,0.000000,1.000000
1994,0.333333,0.250000,0.416667
1995,0.916667,0.000000,0.083333
1996,0.583333,0.166667,0.250000
1997,0.583333,0.333333,0.083333
1998,0.666667,0.250000,0.083333
1999,0.416667,0.166667,0.416667
2000,0.333333,0.583333,0.083333
2001,0.416667,0.416667,0.166667


In [7]:
monthly_df

,Downs,Flats,Ups
Year,,,
1993,0,1,0
1994,3,5,4
1995,0,1,11
1996,2,3,7
1997,4,1,7
1998,3,1,8
1999,2,5,5
2000,7,1,4
2001,5,2,5


In [150]:
contracts = spx_options.loc[[49,17]]
holding_days = 2
positions = [1, 1]
day_format = 'trading'
prem_price_use = 'Mid'

def ev_calc(contracts, positions, holding_days, price_stds = 5, day_format = 'trading', prem_price = 'Mid'):
    vix = curr_stock_data('^VIX')[-1]/100

    if day_format != 'trading':
        year = 365
    else:
        year = 252

    def kelly(odds, p, q):
        return (odds*p - q)/odds

    period_std = vix/np.sqrt(year/holding_days)

    returns = np.linspace(-price_stds*period_std,price_stds*period_std, 300)
    returns = pd.DataFrame({'Returns': returns,
                            'CDF': norm.cdf(returns, 0, period_std)}, index = range(len(returns)))

    returns['probs'] = round(returns['CDF'] - returns['CDF'].shift(1), 6).fillna(0)
    sims = []
    for idx, row in returns.iterrows():
        sims.append(position_sim(contracts, positions, 0, row.Returns, 0, holding_days, prem_price_use = prem_price))

    sim_df = pd.concat(sims,axis = 0)
    sim_df.index = range(len(returns))
    sim_df = sim_df.join(returns)[['PnL','probs','Percent Return']]

    sim_df['EV'] = sim_df.probs*sim_df.PnL
    expected_val = sim_df.EV.sum()
    expected_gain = sim_df[sim_df['EV'] >= 0].EV.sum()
    expected_loss = sim_df[sim_df['EV'] < 0].EV.sum()
    gain_prob = sim_df[sim_df['EV'] >= 0].probs.sum()
    loss_prob = sim_df[sim_df['EV'] < 0].probs.sum()

    out_df = pd.DataFrame({'EV': expected_val,
                           'Expected Win': expected_gain,
                           'Prob Win': gain_prob,
                           'Expected Loss': expected_loss,
                           'Prob Loss': loss_prob,
                           'Net Cost': (contracts.Mid*contracts.Pos).sum()*100,
                           'Kelly Fraction': kelly(expected_gain/((contracts.Mid*contracts.Pos).sum()*100), gain_prob, loss_prob)},
                          index = [0])
    
    return out_df

In [3]:
earnings_history('CRM')

[                Current Qtr Next Qtr Current Year Next Year
 No. of Analysts          35       35           42        42
 Avg. Estimate          0.47     0.54         2.32      2.71
 Low Estimate           0.46     0.46         2.24       2.5
 High Estimate           0.5     0.61         2.41      3.15
 Year Ago EPS           0.33     0.39         1.35      2.32,
            7/30/2017 10/30/2017 1/30/2018 4/29/2018
 EPS Est.        0.32       0.37      0.34      0.46
 EPS Actual      0.33       0.39      0.35      0.74
 Difference      0.01       0.02      0.01      0.28
 Surprise %       3.1        5.4       2.9      60.9,
                  Current Qtr Next Qtr Current Year Next Year
 Current Estimate        0.47     0.54         2.32      2.71
 7 Days Ago              0.47     0.54         2.32      2.71
 30 Days Ago             0.47     0.54         2.32      2.71
 60 Days Ago             0.47     0.54         2.31      2.71
 90 Days Ago             0.48     0.54         2.24      

In [16]:
past_earnings('AMD')

,open,close,AMD
timestamp,,,
2018-07-26,17.16,18.35,0.133921
2018-04-26,10.87,11.04,0.128369
2018-01-31,13.24,13.74,0.065412
2017-10-25,12.95,12.33,-0.144722
2017-07-26,15.13,14.76,0.045037
2017-05-02,11.73,10.32,-0.277456
2017-02-01,10.90,12.06,0.150977
2016-10-21,6.50,6.52,-0.065305
2016-07-22,5.73,5.84,0.112233


In [12]:
### Yahoo query links
'''
Inputs for the ?modules= query:

modules = [
   'assetProfile',
   'incomeStatementHistory',
   'incomeStatementHistoryQuarterly',
   'balanceSheetHistory',
   'balanceSheetHistoryQuarterly',
   'cashflowStatementHistory',
   'cashflowStatementHistoryQuarterly',
   'defaultKeyStatistics',
   'financialData',
   'calendarEvents',
   'secFilings',
   'recommendationTrend',
   'upgradeDowngradeHistory',
   'institutionOwnership',
   'fundOwnership',
   'majorDirectHolders',
   'majorHoldersBreakdown',
   'insiderTransactions',
   'insiderHolders',
   'netSharePurchaseActivity',
   'earnings',
   'earningsHistory',
   'earningsTrend',
   'industryTrend',
   'indexTrend',
   'sectorTrend' ]
Example URL:

https://query1.finance.yahoo.com/v10/finance/quoteSummary/AAPL?modules=assetProfile%2CearningsHistory
Querying for: assetProfile and earningsHistory

'''

ticker = 'AMD'
tickers = 'FB,AMZN'
yahoo_finstatements = "https://query1.finance.yahoo.com/v7/finance/quote?symbols={}&fields=ebitda,shortRatio,priceToSales".format(ticker)
yahoo_finsummary = 'https://query1.finance.yahoo.com/v7/finance/quote?symbols={}'.format(tickers)
yahoo_options = 'https://query1.finance.yahoo.com/v7/finance/options/{}'.format(ticker)

In [57]:
with urlreq.urlopen(yahoo_finsummary) as url:
    data = json.loads(url.read().decode())
    
data

{'quoteResponse': {'error': None,
  'result': [{'ask': 175.9,
    'askSize': 12,
    'averageDailyVolume10Day': 24539500,
    'averageDailyVolume3Month': 24181660,
    'bid': 175.85,
    'bidSize': 8,
    'bookValue': 27.458,
    'currency': 'USD',
    'earningsTimestamp': 1532548800,
    'earningsTimestampEnd': 1541448000,
    'earningsTimestampStart': 1540929600,
    'epsForward': 8.34,
    'epsTrailingTwelveMonths': 6.463,
    'esgPopulated': False,
    'exchange': 'NMS',
    'exchangeDataDelayedBy': 0,
    'exchangeTimezoneName': 'America/New_York',
    'exchangeTimezoneShortName': 'EDT',
    'fiftyDayAverage': 192.12735,
    'fiftyDayAverageChange': -16.227356,
    'fiftyDayAverageChangePercent': -0.08446146,
    'fiftyTwoWeekHigh': 218.62,
    'fiftyTwoWeekHighChange': -42.72,
    'fiftyTwoWeekHighChangePercent': -0.19540757,
    'fiftyTwoWeekLow': 149.02,
    'fiftyTwoWeekLowChange': 26.87999,
    'fiftyTwoWeekLowChangePercent': 0.18037839,
    'fiftyTwoWeekRange': '149.02 - 218

In [177]:
def yahoo_fundamentals(ticker_lst):
    
    for i, ticker in enumerate(ticker_lst):
        with urlreq.urlopen('https://query1.finance.yahoo.com/v10/finance/quoteSummary/{}?modules=assetProfile%2CearningsHistory'.format(ticker)) as url:
            data = json_normalize(json.loads(url.read().decode())['optionChain']['result'][0]['quote']).T
            data.columns = [ticker]
        if i == 0:
            out_df = data
        else:
            out_df = out_df.join(data)
            
    return out_df

In [267]:
# Creating class for querying yahoo data
# yahoo_query(str[ticker], datetime.date([starting_date]), datetime.date([ending_date])):
class yahoo_query:
    
    # Initializing yahoo_query class with ticker and creating
    # relevant URL api calls to query relevant data
    def __init__(self, ticker, start_date, end_date = dt.datetime.today()):
        start_date_unix = int(start_date.timestamp())
        end_date_unix = int(end_date.timestamp())
        
        self.stock_name = ticker
        self.minute_url = 'https://query1.finance.yahoo.com/v8/finance/chart/{0}?symbol={0}&interval=1m'.format(ticker)
        self.hist_price_url = 'https://query1.finance.yahoo.com/v8/finance/chart/{0}?symbol={0}&period1={1}&period2={2}&interval=1d'.format(ticker,start_date_unix,end_date_unix)
        self.options_url = 'https://query1.finance.yahoo.com/v7/finance/options/{}'.format(ticker)
        self.quick_summary_url = 'https://query1.finance.yahoo.com/v7/finance/quote?symbols={}'.format(ticker)
        self.earnings_management_url = 'https://query1.finance.yahoo.com/v10/finance/quoteSummary/{}?modules=assetProfile%2CearningsHistory'.format(ticker)
        self.fin_statements_url = "https://query1.finance.yahoo.com/v7/finance/quote?symbols={}&fields=ebitda,shortRatio,priceToSales".format(ticker)
    
    # Class method for querying yahoo minute data using
    # minute_url defined on initialization
    def minute_query(self):
        with urlreq.urlopen(self.minute_url) as url:
            data = json.loads(url.read().decode())
            self.minute_prices = pd.DataFrame(data['chart']['result'][0]['indicators']['quote'][0],
                                            index = [dt.datetime.utcfromtimestamp(int(x)) - 
                                                     dt.timedelta(hours = 4) for x in 
                                                     data['chart']['result'][0]['timestamp']])
            self.minute_prices.index = pd.to_datetime(self.minute_prices.index)
            self.minute_prices.columns = ["{0}_{1}".format(ticker, x) for x in self.minute_prices.columns]
            
    # Class method for querying yahoo historical prices
    # using hist_price_url on initialization
    def hist_prices_query(self):
        with urlreq.urlopen(self.hist_price_url) as url:
            data = json.loads(url.read().decode())
            self.hist_prices = pd.DataFrame(data['chart']['result'][0]['indicators']['quote'][0],
                                            index = [dt.datetime.utcfromtimestamp(int(x)).date() for x in 
                                                     data['chart']['result'][0]['timestamp']])
            self.hist_prices.index = pd.to_datetime(self.hist_prices.index)
            self.hist_prices.columns = ["{0}_{1}".format(ticker, x) for x in self.hist_prices.columns]
            
    # Class method for querying yahoo earnings data
    # using earnings_management_url on initialization
    def earnings_query(self):
        with urlreq.urlopen(self.earnings_management_url) as url:
            data = json.loads(url.read().decode())
            earnings_history = pd.concat([pd.DataFrame(quarter_earnings).loc['raw'] for 
                                          quarter_earnings in 
                                          data['quoteSummary']['result'][0]['earningsHistory']['history']],
                                         axis = 1).T
            earnings_history.index = pd.to_datetime([dt.datetime.utcfromtimestamp(int(x)).date() for 
                                                     x in earnings_history['quarter'].tolist()])
            self.earnings_history = earnings_history.drop(['period','maxAge','quarter'], axis = 1)
            
            # Additional data figure assignments from same api call
            self.profile = pd.DataFrame(dict((k, data['quoteSummary']['result'][0]['assetProfile'][k]) for 
                                             k in ('industry', 'sector', 'fullTimeEmployees', 'auditRisk', 
                                                   'boardRisk', 'compensationRisk', 'shareHolderRightsRisk', 
                                                   'overallRisk')), index = [ticker])
            
            executives = pd.concat([pd.DataFrame(executive).loc['raw'] for 
                                    executive in data['quoteSummary']['result'][0]['assetProfile']['companyOfficers']],
                                   axis = 1).T
            executives.index = executives.title
            self.executives = executives.drop(['title','maxAge','yearBorn'], axis = 1)
            
        
    # Class method for querying most near-term options chain
    # using  options_url on initialization
    def latest_options_query(self):
        with urlreq.urlopen(self.options_url) as url:
            data = json.loads(url.read().decode())
            options = data['optionChain']['result'][0]['options'][0]
            options = pd.merge(json_normalize(options['calls']), json_normalize(options['puts']), how='inner', on = 'strike',
                               suffixes=('_calls', '_puts'))
            options['expiry'] = dt.datetime.utcfromtimestamp(options['expiration_calls'][0]).date()

            self.options = options.drop(['expiration_calls','expiration_puts','contractSize_calls',
                                         'currency_calls','contractSize_puts','currency_puts',
                                         'lastTradeDate_calls', 'lastTradeDate_puts', 'percentChange_calls',
                                         'percentChange_puts'], axis = 1)
            self.options.index = self.options.strike
            
    # Class method for querying quick quote summary using
    # quick_summary_url on initialzation
    def quick_quote_query(self):
        with urlreq.urlopen(self.quick_summary_url) as url:
            data = json.loads(url.read().decode())
            self.quick_quote = pd.DataFrame(data['quoteResponse']['result'][0], index = [ticker])
            for col in ['dividendDate','earningsTimestamp',
                        'earningsTimestampEnd', 'earningsTimestampStart']:
                self.quick_quote.loc[ticker,col] = dt.datetime.utcfromtimestamp(int(self.quick_quote.loc[ticker,col])).date()
    
    # Class method for querying financials quote summary using
    # fin_statements_url on initialization
    def financials_query(self):
        with urlreq.urlopen(self.fin_statements_url) as url:
            data = json.loads(url.read().decode())
            self.financials = pd.DataFrame(data['quoteResponse']['result'][0], index = [ticker])
            
            
test = yahoo_query('AAPL', dt.datetime(2018,1,1))
test.financials_query()
test.financials

,currency,ebitda,esgPopulated,exchange,exchangeDataDelayedBy,exchangeTimezoneName,exchangeTimezoneShortName,fullExchangeName,gmtOffSetMilliseconds,language,...,quoteType,region,regularMarketPrice,regularMarketTime,revenue,sharesOutstanding,shortRatio,sourceInterval,symbol,tradeable
AAPL,USD,78533001216,False,NMS,0,America/New_York,EDT,NasdaqGS,-14400000,en-US,...,EQUITY,US,222.98,1535572802,2.552740e+11,4829929984,1.61,15,AAPL,True


In [260]:
ticker = 'AAPL'
start_date_unix = int(dt.datetime(2017,1,1).timestamp())
end_date_unix = int(dt.datetime.today().timestamp())
input_url = fin_statements_url = "https://query1.finance.yahoo.com/v7/finance/quote?symbols={}&fields=ebitda,shortRatio,priceToSales".format(ticker)

with urlreq.urlopen(input_url) as url:
    data = json.loads(url.read().decode())

,currency,ebitda,esgPopulated,exchange,exchangeDataDelayedBy,exchangeTimezoneName,exchangeTimezoneShortName,fullExchangeName,gmtOffSetMilliseconds,language,...,quoteType,region,regularMarketPrice,regularMarketTime,revenue,sharesOutstanding,shortRatio,sourceInterval,symbol,tradeable
AAPL,USD,78533001216,False,NMS,0,America/New_York,EDT,NasdaqGS,-14400000,en-US,...,EQUITY,US,222.98,1535572802,2.552740e+11,4829929984,1.61,15,AAPL,True
